In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
image = cv2.imread('images/receipt_image.jpg', 0)

In [3]:
def show(image):
    
    resized_height = 720
    percent = resized_height / len(image)
    resized_width = int(percent * len(image[0]))
    gray = cv2.resize(image,(resized_width,resized_height))


    cv2.imshow('cringe', gray)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Approach 1: Blur, Canny, Threshold, Dilate -> Contours

In [5]:
blurred_image = cv2.GaussianBlur(image, (9,9), 0)
canny_image = cv2.Canny(blurred_image, 50, 150)
threshold_image = cv2.threshold(canny_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
dilated_image = cv2.dilate(threshold_image, None, iterations=5)

In [6]:
show(dilated_image)

# Approach 2: Blur, Gamma, Threshold, Hough -> Contours

In [23]:
blurred_image = cv2.GaussianBlur(image, (9,9), 2)
gamma_image = np.array(255*(blurred_image/255)**3, dtype=np.uint8)

threshold_image = cv2.threshold(canny_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

canny_image = cv2.Canny(gamma_image, 50, 200, None, 3)
lines = cv2.HoughLines(canny_image, 1, np.pi/180, 150, None, 0, 0)

# hough_image = np.zeros_like(image)
hough_image = image.copy()

if lines is not None:
    for i in range(0, len(lines)):
        rho = lines[i][0][0]
        theta = lines[i][0][1]
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
        pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
        cv2.line(hough_image, pt1, pt2, (255, 0, 255), 3, cv2.LINE_AA)
        
show(hough_image)

# maybe crop this image based on hough, then apply clustering, etc.?

# Approach 3: Segmenting based on clustering

In [ ]:
log_image = np.array((255/(np.log(1+np.max(gamma_image)))) * np.log(1+gamma_image), dtype=np.uint8)
# show(log_image)

def cluster(image, k=2):
    pixels = image.astype(np.float32).reshape(-1,1)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)

    _, labels, centers = cv2.kmeans(pixels, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    centers = np.uint8(centers)

    segmented = centers[labels.flatten()]
    segmented = segmented.reshape(image.shape)
    show(segmented)

cluster(log_image, 2)

# Approach 4: Segmenting based on Colors

In [ ]:
color_image = cv2.imread('receipt_image.jpg')
hsv_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2HSV)

lower_red = np.array([0, 100, 100])
upper_red = np.array([10, 255, 255])
mask1 = cv2.inRange(hsv_image, lower_red, upper_red)

# Another range for red (because red wraps around the hue spectrum)
lower_red2 = np.array([160, 100, 100])
upper_red2 = np.array([180, 255, 255])
mask2 = cv2.inRange(hsv_image, lower_red2, upper_red2)

# Combine both masks
mask = mask1 | mask2

# Step 4: Apply the mask to the original image
segmented_image = cv2.bitwise_and(image, image, mask=mask)

show(hsv_image)
print(hsv_image[480,480])

# Approach 5: Trial

In [22]:
color_image = cv2.imread('receipt_image.jpg')
# hsv_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2HSV)

blurred_image = cv2.GaussianBlur(color_image, (9,9), 0)

hsv_image = cv2.cvtColor(blurred_image, cv2.COLOR_BGR2HSV)
# adaptive_thresholded = cv2.adaptiveThreshold(blurred_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
# show(adaptive_thresholded)
show(hsv_image)



In [14]:
blurred_image = cv2.GaussianBlur(image, (15,15), 5)
rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 15))
dilated = cv2.dilate(blurred_image, rectKernel, iterations=3)

show(dilated)

In [20]:
# different types of blurred images
show(cv2.bilateralFilter(image, 15, 75, 75))
# show(cv2.medianBlur(image, 9))


# Finding the Contours

In [10]:
contours, _ = cv2.findContours(log_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cnt_image = image.copy()
cv2.drawContours(cnt_image, contours, -1, (0,255,0), 3)
rect_contours = []

for contour in contours:
    peri = cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, 0.02*peri, True)
    
    if len(approx) == 4:
        rect_contours.append(approx)

In [11]:
show(cnt_image)

In [21]:
show(image)

In [25]:
test = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
lines = cv2.HoughLines(canny_image, 1, np.pi/180, 150, None, 0, 0)

# hough_image = np.zeros_like(image)
hough_image = image.copy()

if lines is not None:
    for i in range(0, len(lines)):
        rho = lines[i][0][0]
        theta = lines[i][0][1]
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
        pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
        cv2.line(hough_image, pt1, pt2, (255, 0, 255), 3, cv2.LINE_AA)